In [2]:
import pandas as pd
import json

Inception v3 trained on balanced dataset, focal loss $\gamma = 2$, 5 epochs, 100 batch size, Adam lr=0.0005

In [3]:
# path = "../results/inception_v3/fl/"
path = "../results/inception_v3/xen/"
# path = "../results/AlexNet/xen/"
# path = "../results/AlexNet/xen_2/"

In [4]:
df = pd.read_csv(path+"avg_metrics.csv")

In [5]:
df.columns

Index(['image_idxs', 'accuracy', 'precision', 'specificity', 'sensibility',
       'IoU', 'dice_coeff', 'f1_score'],
      dtype='object')

In [6]:
# Function to parse indices string into list of ints
def parse_json_column(json_str):
    try:
        return json.loads(json_str)
    except (TypeError, json.JSONDecodeError):
        return None

# Apply the parsing function to the 'data' column
df["image_idxs"] = df["image_idxs"].apply(parse_json_column)

In [7]:
df

,image_idxs,accuracy,precision,specificity,sensibility,IoU,dice_coeff,f1_score
0,"[0, 1, 2]",0.918296,0.953550,0.841486,0.937204,0.896572,0.945208,0.893532
1,"[3, 4, 5]",0.865958,0.934580,0.807934,0.878138,0.827613,0.905372,0.865557
2,"[6, 7, 8]",0.961393,0.973876,0.596367,0.983999,0.958744,0.978903,0.725794
3,"[9, 10, 11]",0.933863,0.947225,0.704407,0.975952,0.925551,0.961333,0.807089
4,"[12, 13, 14]",0.938709,0.969998,0.688468,0.961000,0.933423,0.965473,0.800992
5,"[15, 16, 17]",0.883914,0.887994,0.672692,0.944407,0.845602,0.914879,0.758147
6,"[18, 19, 20]",0.950076,0.951141,0.612103,0.994076,0.945828,0.972067,0.742960
7,"[21, 22, 23]",0.806899,0.794264,0.603529,0.875334,0.763686,0.864898,0.681789
8,"[24, 25, 26]",0.868155,0.786452,0.513866,0.922327,0.854315,0.920304,0.621588
9,"[27, 28]",0.914234,0.923230,0.549353,0.976260,0.902922,0.948919,0.669523


In [8]:
metrics_df = df.drop(columns=['image_idxs'])

In [9]:
cols = metrics_df.columns
n_samples, n_features = metrics_df.shape

In [10]:
list(cols)

['accuracy',
 'precision',
 'specificity',
 'sensibility',
 'IoU',
 'dice_coeff',
 'f1_score']

In [11]:
means = metrics_df.mean()
means

accuracy       0.904150
precision      0.912231
specificity    0.659021
sensibility    0.944870
IoU            0.885426
dice_coeff     0.937736
f1_score       0.756697
dtype: float64

In [12]:
non_nan_counts = metrics_df.count()
stds = metrics_df.std()/non_nan_counts
stds

accuracy       0.004759
precision      0.006874
specificity    0.010588
sensibility    0.004219
IoU            0.006154
dice_coeff     0.003584
f1_score       0.008672
dtype: float64

In [13]:
n_decimals = 4
cross_val_data = [f"{means.iloc[i]:0.{n_decimals}f} +- {stds.iloc[i]:0.{n_decimals}f}" for i in range(n_features)]

In [14]:
cross_val_metrics_df = pd.DataFrame([cross_val_data], columns=tuple(cols))
cross_val_metrics_df.loc[0]

accuracy       0.9041 +- 0.0048
precision      0.9122 +- 0.0069
specificity    0.6590 +- 0.0106
sensibility    0.9449 +- 0.0042
IoU            0.8854 +- 0.0062
dice_coeff     0.9377 +- 0.0036
f1_score       0.7567 +- 0.0087
Name: 0, dtype: object